# O que faremos nesse notebook

Na primeira etapa dos experimentos de finetuning, nós fizemos um total de 20 experimentos, variando:
 - tamanho do modelo (`t5-base` e `t5-small`)
 - como tratar caracteres `\n`: substituindo por espaço (` `) ou pipes (`|`)
 - modelo treinado sobre dataset com apenas uma tarefa ou todas (concatenação de datasets de todas tarefas)

Aqui iremos escolher, para cada tarefa, qual foi a melhor combinação de configurações.
Iremos retreinar os melhores modelos, usando o número de epochs do melhor checkpoint, usando todos os dados rotulados disponíveis (`train` e `test`). Os modelos resultantes dessa etapa serão usados para gerar os dados que iremos submeter no site da competição.

# Imports and initial setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from src.data.sroie import get_all_keynames_from_dir, load_labels
from tqdm import tqdm
import gc
import gin
from src.evaluation.sroie_eval_utils import extract_base_keyname
import multiprocessing as mp
import os
import torch
import pandas as pd
from src.metrics import compute_precision_recall_f1 # This is word level
from src.metrics import compute_exact
import numpy as np
import neptune

gin.enter_interactive_mode()
gin.clear_config()

gin.enter_interactive_mode()
gin.clear_config()

In [3]:
# Neptune data
project = neptune.init('marcospiau/final-project-ia376j-1')

finetuning_exp_ids = [
    'FIN-27', 'FIN-28', 'FIN-29', 'FIN-30', 'FIN-33', 'FIN-34', 'FIN-35',
    'FIN-36', 'FIN-37', 'FIN-38', 'FIN-39', 'FIN-40', 'FIN-41', 'FIN-42', 
    'FIN-43', 'FIN-44', 'FIN-45', 'FIN-46', 'FIN-47', 'FIN-48'
]

experiments = dict(
    (x.id, x)
    for x in project.get_experiments(finetuning_exp_ids)
)

In [4]:
# Same data configs used on finetuning
gin.parse_config_file(
    '/home/marcospiau/final_project_ia376j/src/models/gin/defaults.gin',
    skip_unknown=True
)
with gin.config_scope('train_sroie'):
    train_keynames = get_all_keynames_from_dir()

with gin.config_scope('validation_sroie'):
    val_keynames = get_all_keynames_from_dir()

gin.clear_config()

In [5]:
predictions_basedir = "/home/marcospiau/final_project_ia376j/data/sroie_receipt_dataset/predictions"

def get_keynames_from_experiment(
    experiment_id,
    partition,
    predictions_basedir=predictions_basedir
):
    keynames_path = os.path.join(predictions_basedir, experiment_id, partition)
    return get_all_keynames_from_dir(keynames_path)

def get_labels_dict(keynames):
    """Loads labels (or predictions) for `keynames`.txt.
    """
    return {extract_base_keyname(x): load_labels(x) for x in keynames}

def compute_metric_on_dict_keys(func, true, pred):
    """Computes metrics for dicts with matching keys. All inner dicts must have
    the same keys.
    """
    assert true.keys() == pred.keys()
    return {k: func(true[k], pred[k]) for k in true}

def reduce_metrics_dict(x, reduce_func):
    """Reduces metrics on dict. All inner dicts must have the same keys.
    """
#     dropping first level keys (sroie keymames)
    x = list(x.values())
    field_keys = list(x[0].keys())
    metric_keys = list(x[0][field_keys[0]].keys())
    
    broadcast_dict = {
        field_key: {
            metric_key:[]
            for metric_key in metric_keys
        }
        for field_key in field_keys
    }
    for example in x:
        for field_key in field_keys:
            for metric_key in metric_keys:
                broadcast_dict[field_key][metric_key].append(
                    example[field_key][metric_key]
                )
    reduced_dict = {
        field_key: {
            metric_key: reduce_func(broadcast_dict[field_key][metric_key])
            for metric_key in metric_keys
        }
        for field_key in field_keys
    }
#     return broadcast_dict
    return reduced_dict


def compute_metrics_by_fields(experiment_id, partition, true_labels_dict):
    preds_dict = get_labels_dict(
        get_keynames_from_experiment(experiment_id, partition)
    )
    assert true_labels_dict.keys() == preds_dict.keys()
    _compute_exact = lambda a_gold, a_pred: {
        'exact': compute_exact(a_gold, a_pred)}
#     metrics per example
    exact_all_examples = {
        k: compute_metric_on_dict_keys(
            _compute_exact, true_labels_dict[k], preds_dict[k]
        )
        for k in preds_dict
    }   
    precision_recall_f1_all_examples = {
        k: compute_metric_on_dict_keys(
            compute_precision_recall_f1, true_labels_dict[k], preds_dict[k]
        )
        for k in preds_dict
    }
#     merge all metrics
    fields = next(iter(preds_dict.values())).keys()
    merge_metrics = {
        keyname: {
            field: {
                **precision_recall_f1_all_examples[keyname][field],
                **exact_all_examples[keyname][field]
            }
            for field in fields
        }
        for keyname in preds_dict
    }
#     Reduce metrics by mean
    reduced_metrics = reduce_metrics_dict(merge_metrics, np.mean)
    
    return reduced_metrics

In [6]:
# Ground truth predictions: same for all experiments
true_labels = {
    'train': get_labels_dict(train_keynames),
    'test': get_labels_dict(val_keynames)
}

Metrics for all experiments:

In [7]:
val_results = pd.concat([
    pd.DataFrame(
        compute_metrics_by_fields(exp_id, 'test', true_labels['test'])
    ).assign(exp_id = exp_id)
    for exp_id in tqdm(experiments.keys())
])
val_results['all_tasks_concat'] = val_results[
    ['company', 'date', 'address', 'total']
].mean(1)
val_results = val_results.rename(
    {x: f'extract_{x}' for x in ['company', 'date', 'address', 'total']}, axis=1
)
val_results['exp_id'] = val_results.pop('exp_id')
val_results = val_results.reset_index().rename({'index': 'eval_metric'}, axis=1)
val_results = val_results.set_index('eval_metric')

val_results.head()

100%|██████████| 20/20 [00:02<00:00,  9.42it/s]


,extract_company,extract_date,extract_address,extract_total,all_tasks_concat,exp_id
eval_metric,,,,,,
precision,0.930298,0.975,0.902343,0.895000,0.925660,FIN-27
recall,0.914119,0.975,0.889215,0.907500,0.921458,FIN-27
f1,0.918484,0.975,0.894129,0.898333,0.921487,FIN-27
exact,0.820000,0.945,0.440000,0.875000,0.770000,FIN-27
precision,0.883796,0.995,0.897181,0.940000,0.928994,FIN-28


In [8]:
exp_properties = (
    pd.DataFrame({k:v.get_properties() for k,v in tqdm(experiments.items())})
    .T
    .reset_index()
    .rename({'index': 'exp_id'},axis=1)
    [['exp_id', 'task_train', 'param__t5_model_prefix', 'str_replace_newlines']]
    .rename({'param__t5_model_prefix': 'param__t5_model_prefix'}, axis=1)
)
exp_properties['exp_name'] = exp_properties['exp_id'].map(
    lambda x: experiments[x].name)
exp_properties

100%|██████████| 20/20 [00:07<00:00,  2.68it/s]


,exp_id,task_train,param__t5_model_prefix,str_replace_newlines,exp_name
0,FIN-27,all_tasks_concat,t5-base,|,t5-base_all_tasks_concat_newlines_as_pipes
1,FIN-28,all_tasks_concat,t5-base,,t5-base_all_tasks_concat_newlines_as_spaces
2,FIN-29,all_tasks_concat,t5-small,|,t5-small_all_tasks_concat_newlines_as_pipes
3,FIN-30,all_tasks_concat,t5-small,,t5-small_all_tasks_concat_newlines_as_spaces
4,FIN-33,extract_company,t5-base,|,t5-base_extract_company_newlines_as_pipes
5,FIN-34,extract_company,t5-base,,t5-base_extract_company_newlines_as_spaces
6,FIN-35,extract_company,t5-small,|,t5-small_extract_company_newlines_as_pipes
7,FIN-36,extract_company,t5-small,,t5-small_extract_company_newlines_as_spaces
8,FIN-37,extract_total,t5-base,|,t5-base_extract_total_newlines_as_pipes
9,FIN-38,extract_total,t5-base,,t5-base_extract_total_newlines_as_spaces


Merging both:

In [9]:
val_results = (
    val_results
    .merge(right = exp_properties, how='left', on='exp_id')
    .set_index(val_results.index)
)
val_results.head()

,extract_company,extract_date,extract_address,extract_total,all_tasks_concat,exp_id,task_train,param__t5_model_prefix,str_replace_newlines,exp_name
eval_metric,,,,,,,,,,
precision,0.930298,0.975,0.902343,0.895000,0.925660,FIN-27,all_tasks_concat,t5-base,|,t5-base_all_tasks_concat_newlines_as_pipes
recall,0.914119,0.975,0.889215,0.907500,0.921458,FIN-27,all_tasks_concat,t5-base,|,t5-base_all_tasks_concat_newlines_as_pipes
f1,0.918484,0.975,0.894129,0.898333,0.921487,FIN-27,all_tasks_concat,t5-base,|,t5-base_all_tasks_concat_newlines_as_pipes
exact,0.820000,0.945,0.440000,0.875000,0.770000,FIN-27,all_tasks_concat,t5-base,|,t5-base_all_tasks_concat_newlines_as_pipes
precision,0.883796,0.995,0.897181,0.940000,0.928994,FIN-28,all_tasks_concat,t5-base,,t5-base_all_tasks_concat_newlines_as_spaces


In [10]:
pd.options.display.max_rows = 500
pd.options.display.max_columns = None

def get_sort_dict(sorted_values):
    """Returns a dict where the values are sorted acording keys specification.
    """
    return dict((k,v) for v,k in enumerate(sorted_values))
    
def choose_best_model(df, task_eval, metric_eval, show_top_k_models=None):
#     Getting best experiments
    df = df.copy()
    sorted_exps = df.loc[metric_eval].sort_values(task_eval, ascending=False)
    
    exp_ord_dict = get_sort_dict(sorted_exps['exp_id'])
    df['exp_rank'] = df['exp_id'].map(exp_ord_dict.get)
    
    metric_ord_dict = get_sort_dict(['precision', 'recall', 'f1', 'exact'])
    df['tmp_ord_metric'] = df.index.to_series().map(metric_ord_dict.get)
    df = (
        df
        .reset_index()
        .sort_values(['exp_rank', 'tmp_ord_metric'])
        .drop('tmp_ord_metric', axis=1)
        .set_index(['exp_name', 'exp_id', 'exp_rank', 'eval_metric'])
    )
    if show_top_k_models is not None:
        df = df.groupby(level=-1).head(show_top_k_models)
    df = df.style.set_properties(
        **{'background-color': 'lightgreen'},subset=task)
    return df

All models sorted by exp_id (same on report appendice):

In [39]:
display(
    choose_best_model(val_results, 'all_tasks_concat', 'f1')
    .data
    .reset_index()
    .drop('exp_rank', axis=1)
    .sort_values('exp_id')
    .reset_index(drop=True)
)

,exp_name,exp_id,eval_metric,extract_company,extract_date,extract_address,extract_total,all_tasks_concat,task_train,param__t5_model_prefix,str_replace_newlines
0,t5-base_all_tasks_concat_newlines_as_pipes,FIN-27,precision,0.930298,0.975000,0.902343,0.895000,0.925660,all_tasks_concat,t5-base,|
1,t5-base_all_tasks_concat_newlines_as_pipes,FIN-27,recall,0.914119,0.975000,0.889215,0.907500,0.921458,all_tasks_concat,t5-base,|
2,t5-base_all_tasks_concat_newlines_as_pipes,FIN-27,f1,0.918484,0.975000,0.894129,0.898333,0.921487,all_tasks_concat,t5-base,|
3,t5-base_all_tasks_concat_newlines_as_pipes,FIN-27,exact,0.820000,0.945000,0.440000,0.875000,0.770000,all_tasks_concat,t5-base,|
4,t5-base_all_tasks_concat_newlines_as_spaces,FIN-28,precision,0.883796,0.995000,0.897181,0.940000,0.928994,all_tasks_concat,t5-base,
5,t5-base_all_tasks_concat_newlines_as_spaces,FIN-28,recall,0.876042,0.995000,0.886881,0.932500,0.922606,all_tasks_concat,t5-base,
6,t5-base_all_tasks_concat_newlines_as_spaces,FIN-28,f1,0.873571,0.995000,0.890402,0.933333,0.923077,all_tasks_concat,t5-base,
7,t5-base_all_tasks_concat_newlines_as_spaces,FIN-28,exact,0.720000,0.995000,0.410000,0.910000,0.758750,all_tasks_concat,t5-base,
8,t5-small_all_tasks_concat_newlines_as_pipes,FIN-29,f1,0.896546,0.973333,0.889479,0.925833,0.921298,all_tasks_concat,t5-small,|
9,t5-small_all_tasks_concat_newlines_as_pipes,FIN-29,exact,0.730000,0.940000,0.430000,0.900000,0.750000,all_tasks_concat,t5-small,|


Best model for each task:

In [11]:
for task in exp_properties.task_train.unique():
    print(120 * '*')
    print(f'Best model for task {task}')
    display(choose_best_model(val_results, task, 'f1', 1))

************************************************************************************************************************
Best model for task all_tasks_concat


************************************************************************************************************************
Best model for task extract_company


************************************************************************************************************************
Best model for task extract_total


************************************************************************************************************************
Best model for task extract_address


************************************************************************************************************************
Best model for task extract_date


In [13]:
best_exp_ids = {
    task: (
        choose_best_model(val_results, task, 'f1', 1)
        .data
        .index
        .get_level_values('exp_id').unique()[0]
    )
    for task in exp_properties.task_train.unique()
}

best_exp_names = {
    task: (
        choose_best_model(val_results, task, 'f1', 1)
        .data
        .index
        .get_level_values('exp_name').unique()[0]
    )
    for task in exp_properties.task_train.unique()
}

print(f'best_exp_ids -> {best_exp_ids}')
print(f'best_exp_names -> {best_exp_names}')

best_exp_ids -> {'all_tasks_concat': 'FIN-28', 'extract_company': 'FIN-27', 'extract_total': 'FIN-38', 'extract_address': 'FIN-41', 'extract_date': 'FIN-28'}
best_exp_names -> {'all_tasks_concat': 't5-base_all_tasks_concat_newlines_as_spaces', 'extract_company': 't5-base_all_tasks_concat_newlines_as_pipes', 'extract_total': 't5-base_extract_total_newlines_as_spaces', 'extract_address': 't5-base_extract_address_newlines_as_pipes', 'extract_date': 't5-base_all_tasks_concat_newlines_as_spaces'}


In [14]:
' '.join(set(best_exp_names.values()))

't5-base_extract_address_newlines_as_pipes t5-base_all_tasks_concat_newlines_as_spaces t5-base_all_tasks_concat_newlines_as_pipes t5-base_extract_total_newlines_as_spaces'

In [15]:
[print(x) for x in set(best_exp_names.values())]

t5-base_extract_address_newlines_as_pipes
t5-base_all_tasks_concat_newlines_as_spaces
t5-base_all_tasks_concat_newlines_as_pipes
t5-base_extract_total_newlines_as_spaces


[None, None, None, None]

What we will need to run:

In [16]:
{(best_exp_ids[k], best_exp_names[k]) for k in best_exp_ids.keys()}

{('FIN-27', 't5-base_all_tasks_concat_newlines_as_pipes'),
 ('FIN-28', 't5-base_all_tasks_concat_newlines_as_spaces'),
 ('FIN-38', 't5-base_extract_total_newlines_as_spaces'),
 ('FIN-41', 't5-base_extract_address_newlines_as_pipes')}

Generating final gin files:

In [17]:
cmd_to_format = """
cat ./t5_default_finetune/{exp_name}.gin \\
| sed s,defaults.gin,best_t5_models_defaults.gin,g \\
| sed s,'early_stop.patience = 50','early_stop.patience = 99999',g \\
| sed s,'trainer.min_epochs = 20','trainer.min_epochs = 1',g \\
| sed s,'trainer.max_epochs = 200','trainer.max_epochs = {max_epochs}',g \\
| sed s,"neptune_logger.experiment_name = '{exp_name}'","neptune_logger.experiment_name = '{new_exp_name}'",g \\
> ./t5_best_models_finetune/{new_exp_name}.gin
"""

# Com certeza tem jeitos melhores de fazer isso, mas eu estava com pressa
def extract_max_epochs_from_checkpoint(x):
    
    basename = os.path.basename(x)
    start = basename.find('epoch=')
    end = basename.find('val_precision=')
    return int(basename[start+len('epoch')+1:end-1]) + 1

for exp_id, exp_name in {(best_exp_ids[k], best_exp_names[k]) for k in best_exp_ids.keys()}:
    checkpoint_file = experiments[exp_id].get_channels()['best_model_path'].y
#     print(checkpoint_file)
    print(cmd_to_format.format(**{
        'exp_name': exp_name,
        'max_epochs': extract_max_epochs_from_checkpoint(checkpoint_file),
        'new_exp_name': f'all_labelled_data-{exp_id}-{exp_name}'
    }))
#     break
    
    


cat ./t5_default_finetune/t5-base_extract_address_newlines_as_pipes.gin \
| sed s,defaults.gin,best_t5_models_defaults.gin,g \
| sed s,'early_stop.patience = 50','early_stop.patience = 99999',g \
| sed s,'trainer.min_epochs = 20','trainer.min_epochs = 1',g \
| sed s,'trainer.max_epochs = 200','trainer.max_epochs = 7',g \
| sed s,"neptune_logger.experiment_name = 't5-base_extract_address_newlines_as_pipes'","neptune_logger.experiment_name = 'all_labelled_data-FIN-41-t5-base_extract_address_newlines_as_pipes'",g \
> ./t5_best_models_finetune/all_labelled_data-FIN-41-t5-base_extract_address_newlines_as_pipes.gin


cat ./t5_default_finetune/t5-base_all_tasks_concat_newlines_as_spaces.gin \
| sed s,defaults.gin,best_t5_models_defaults.gin,g \
| sed s,'early_stop.patience = 50','early_stop.patience = 99999',g \
| sed s,'trainer.min_epochs = 20','trainer.min_epochs = 1',g \
| sed s,'trainer.max_epochs = 200','trainer.max_epochs = 22',g \
| sed s,"neptune_logger.experiment_name = 't5-base_all